In [ ]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
df1 = pd.read_csv('./resources/storeid_201908251010.csv', ',')
df2 = pd.read_csv('./resources/targets_201908251011.csv', ',')

df3 = pd.merge(df1, df2, on='id', how='outer')

missing_api_values = df3[df3['storeid'] == 'API']
missing_website_values = df3[df3['storeid'] == 'WEBSITE']
# df3 = df3.set_index('id')
# missing_api_values = missing_api_values.head(100)
# missing_api_values
# missing_api_values.shape

In [ ]:
# get values from API

In [ ]:
missing_api_values.shape[0]

In [ ]:
my_list = []
current_line = 1
cap_size = 1000
warn_size = 20

for index, row in missing_api_values.iterrows():
    
    if current_line % warn_size == 0:
        print(current_line)
        
        if current_line % cap_size == 0:
            create_csv(my_list,'missing_api_values_'+ str(current_line) +'.csv')
            my_list = []
        
    current_line += 1
    
    base_url = 'https://y29rdnycjd.execute-api.eu-west-1.amazonaws.com/dev/missingdata/'
    search_for = row.id

    my_missing_opts = requests.get(base_url + search_for)
    
    if my_missing_opts.status_code != 404:
        my_missing_opts = my_missing_opts.json()
        
        my_missing_opts['returned'] = row.returned
        my_list.append(my_missing_opts)

    else:
        print('not found')
        
# missing_api_values.head()

# my_list

In [ ]:
list_of_keys = ['id','orderportalid','orderdate_gmt','designer','style','shipper','shiptypeid','userid','isvip','country','region','ddprate','countrycode','hasusedwishlist','isreseller','hasitemsonbag','tierafterorder','tierbeforeorder','isusingmultipledevices','userfraudstatus','promocode','freereturn','issale','productid','brand','ddpsubcategory','storeid','countryoforigin','size','category_1stlevel','platform']
my_dict_list = []
my_dict = {}

page = 1

while True:
    try:
        link = 'https://scrap-me.herokuapp.com/items?page=' + str(page)
        r = requests.get(link)
        data = r.text
        soup = BeautifulSoup(data)
        rows = soup.find("tbody").find_all("tr")
        page += 1
        
        for row in rows:
            cells = row.find_all("td")
            for index, feature in enumerate(cells):
                my_dict[list_of_keys[index]] = feature.get_text()
            my_dict_list.append(my_dict)
            my_dict = {}
            
    except Exception as ex:
        print(ex)
        break # exit `while` loop

# print(my_dict_list)
create_csv(my_dict_list)
#         print()
#     print('id', 'orderportalid', 'orderdate_gmt', 'designer', 'style', 'shipper', 'shiptypeid', 'userid', 'isvip', 'country', 'region', 'ddprate', 'countrycode', 'hasusedwishlist', 'isreseller', 'hasitemsonbag', 'tierafterorder', 'tierbeforeorder', 'isusingmultipledevices', 'userfraudstatus 	promocode 	freereturn 	issale 	productid 	brand 	ddpsubcategory 	storeid 	countryoforigin 	size 	category_1stlevel 	platform')

In [ ]:
def create_csv(data, name):
    """ returns (file_basename, server_path, file_size) """
    file_basename = name
    server_path = './resources/'
    w_file = open(server_path + file_basename, 'w', newline='')
    keys = data[0].keys()

    with w_file as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)

    w_file.close()

    w_file = open(server_path + file_basename, 'r')
    file_size = len(w_file.read())
    return file_basename, server_path, file_size

In [ ]:
md_1 = pd.read_csv('./resources/missing_api_values_1000.csv', ',')

for x in range(2,11):
    new_pd = pd.read_csv('./resources/missing_api_values_'+ str(x) +'000.csv', ',')
    md_1 = md_1.append(new_pd)
    
# md_1.drop(columns=['returned'])
md_1.to_csv(r'resources\full_api.csv', index=False)
# md_1

In [ ]:
api = pd.read_csv('./resources/full_api.csv', ',')
# api = api.drop(columns=['returned'])
web = pd.read_csv('./resources/missing_website_values.csv', ',')
# api = api.append(web)
# api.shape

In [ ]:
df1 = pd.read_csv('./resources/storeid_201908251010.csv', ',')
df2 = pd.read_csv('./resources/targets_201908251011.csv', ',')

df3 = pd.merge(df1, df2, on='id', how='outer')
missing_website_values = df3[df3['storeid'] == 'API']
missing_website_values

In [ ]:
final = pd.merge(df3, df2, on='id', how='outer')

In [ ]:
missing_website_values = df3[df3['storeid'] == 'WEBSITE']
missing_website_values.head()

In [ ]:
web.head()

In [ ]:
df3 = df3.drop(columns=['storeid'])
final_web = pd.merge(df3, web, on='id', how='inner')
final_web.to_csv(r'resources\fullweb.csv', index=False)
final_web.head()

In [ ]:
api_f = pd.read_csv('./resources/full_api.csv', ',')
web_f = pd.read_csv('./resources/fullweb.csv', ',')
web_f = web_f.drop(columns=['id'])
web_f = web_f.append(api_f)
web_f.to_csv(r'resources\full_final.csv', index=False)

In [ ]:
final_web[final_web['id'] == 'a0d86abe2f9a44abecbabbef612c90c4']

In [ ]:
api.head()

In [ ]:
final_web_api = pd.merge(final_web, api, on='id', how='outer')

In [ ]:
final_web = final_web.to_csv(r'resources\fullweb.csv', index=False)

In [ ]:
print(final_web)